In [1]:
%load_ext autoreload
%autoreload 2

import datetime
import glob
import pandas as pd
import time
import qfutils
from functools import reduce
from quantfin import DataFeeder, DataBuilder


chrome_driver = r'C:\Users\baoloc.lam\1-LocLam\PythonProjects\chromedriver.exe'

In [ ]:
# # =============================================================================
# #  ########## Mass download stock data from CP68 ##########

# feeder = DataFeeder()
# symbols = 'all'
# login_username = 'loc.lam92@gmail.com'
# login_password = 'homesweethome92'
# data = 'all'
# to_folder_path = 'C:/Users/baoloc.lam/1-LocLam/PythonProjects/vn_stock_trading/raw_data'

# start = datetime.datetime.now()
# feeder.cp68_mass_download(
#     symbols=symbols,
#     login_usernam=login_username,
#     login_password=login_password,
#     data=data,
#     to_folder=to_folder_path
# )
# print('Process time:', datetime.datetime.now() - start)

# # =============================================================================

In [ ]:
# # =============================================================================
# #  ########## Join all raw data to one table save to new file ##########

# from_folder_path = r'C:\Users\baoloc.lam\1-LocLam\PythonProjects\vn_stock_trading\raw_data'
# xls_files = glob.glob('/'.join([from_folder_path, 'excelfull_*.txt']))
# mt4_files = glob.glob('/'.join([from_folder_path, 'metastock_*.txt']))

# to_folder_path = r'C:\Users\baoloc.lam\1-LocLam\PythonProjects\vn_stock_trading\processed_data'
# filepath_xls = '/'.join([to_folder_path, 'xls_data.pkl'])
# filepath_mt4 = '/'.join([to_folder_path, 'mt4_data.pkl'])

# idx = ['symbol', 'date']
# columns_dict = {
#     '<Ticker>': 'symbol',
#     '<DTYYYYMMDD>': 'date',
#     '<OpenFixed>': 'adj_open',
#     '<HighFixed>': 'adj_high',
#     '<LowFixed>': 'adj_low',
#     '<CloseFixed>': 'adj_close',
#     '<Volume>': 'vol',
#     '<Open>': 'open',
#     '<High>': 'high',
#     '<Low>': 'low',
#     '<Close>': 'close',
#     '<VolumeDeal>': 'vol_deal',
#     '<VolumeFB>': 'vol_fb',
#     '<VolumeFS>': 'vol_fs'
# }

# xls_data = pd.DataFrame()
# mt4_data = pd.DataFrame()

# print('Loading file...')

# # Excel data
# print('Loading all Excel file to one single DataFrame object...')
# xls_data = pd.concat((pd.read_csv(file) for file in xls_files))
# xls_data.rename(columns=columns_dict, inplace=True)
# xls_data[idx[1]] = pd.to_datetime(xls_data[idx[1]].astype('str'), format='%Y%m%d')

# print('Re-indexing the DataFrame and save it to new folder...')
# xls_data.set_index(idx, inplace=True)
# xls_data.sort_index(level=idx, inplace=True)
# xls_data.to_pickle(filepath_xls)

# print('All Excel data are loaded to one DataFrame and save to a new folder!')

# # MT4 data
# print('Loading MT4 files to one single DataFrame object...')
# mt4_data = pd.concat((pd.read_csv(file) for file in mt4_files))
# mt4_data.rename(columns=columns_dict, inplace=True)
# mt4_data[idx[1]] = pd.to_datetime(mt4_data[idx[1]].astype('str'), format='%Y%m%d')

# print('Re-indexing the DataFrame and save it to new folder...')
# mt4_data.set_index(idx, inplace=True)
# mt4_data.sort_index(level=idx, inplace=True)
# mt4_data.to_pickle(filepath_mt4)

# print('All MT4 data are loaded to one DataFrame and save to a new folder!')

# print('Data from Excel\n', xls_data)
# print('='*79)
# print('Data from MT4\n', mt4_data)

# # =============================================================================

In [ ]:
# # =============================================================================
#  ########## Direct extract data from web ##########

# feeder = DataFeeder()

# folderpath = r'C:\Users\baoloc.lam\1-LocLam\PythonProjects\vn_stock_trading\processed_data'
# symbol_filename = 'symbols.txt'
# symbol_filepath = '/'.join([folderpath, symbol_filename])

# with open(symbol_filepath, 'r') as f:
#     symbols = f.read()

# symbols = list(symbols.split('\n'))
# login_username = 'loc.lam92@gmail.com'
# login_password = 'homesweethome92'
# cp68_data_source = 'mt4'

# vnd_data = feeder.multithreading(
#     methods='vnd',
#     symbols_array=symbols,
#     date_format=None,
#     records=None,
#     workers=5
# )

# cp68_data = feeder.multithreading(
#     methods='cp68',
#     symbols_array=symbols,
#     date_format=None,
#     records=None,
#     cp68_username=login_username,
#     cp68_password=login_password,
#     cp68_data_source=cp68_data_source,
#     workers=10
# )

# cp68_data = cp68_data[0]['close_mt4', 'vol_mt4']
# vnd_data = vnd_data[1]['adj_close_vnd', 'vol_vnd']

# txn_data = cp68_data.join(vnd_data, how='outer')
# txn_data = DataBuilder(txn_data)
# txn_data.fill_na(
#     'close_mt4',
#     txn_data.data.loc[:, 'adj_close_vnd'],
#     inplace=True
# )
# txn_data.fill_na(
#     'vol_mt4',
#     txn_data.data.loc[:, 'vol_vnd'],
#     inplace=True
# )
# txn_data.data.drop(
#     columns=['adj_close_vnd', 'vol_vnd'],
#     inplace=True
# )
# txn_data.data.columns = ['adj_close', 'vol']

# txn_filepath = '/'.join([folderpath, 'txn_data2.pkl'])
# txn_data.data.to_pickle(txn_filepath)

# # =============================================================================

In [ ]:
# ===============================================================================
#  ########## Update data ##########

folderpath = r'C:\Users\baoloc.lam\1-LocLam\PythonProjects\vn_stock_trading\processed_data'
txn_filename = 'txn_data.pkl'
txn_filepath = '/'.join([folderpath, txn_filename])

txn_data = pd.read_pickle(txn_filepath)
data = DataBuilder(txn_data)

# login_username = 'loc.lam92@gmail.com'
# login_password = 'homesweethome92'
# data_source = 'mt4'

# data.update(
#     method='cp68',
#     records=20,
#     cp68_username=login_username,
#     cp68_password=login_password,
#     cp68_data_source=data_source,
#     workers=20
# )

data.update(
    method='vnd',
    records=20,
    workers=10
)

# data.fill_na('adj_close', data.data['close_mt4'], inplace=True)
# data.fill_na('adj_close', data.data['adj_close_vnd'], inplace=True)
# data.fill_na('vol', data.data['vol_mt4'], inplace=True)
# data.fill_na('vol', data.data['vol_vnd'], inplace=True)

# holidays_filename = 'vn_public_holidays.csv'
# holidays_filepath = '/'.join([folderpath, holidays_filename])
# holidays = pd.read_csv(holidays_filepath)
# holidays = pd.to_datetime(holidays['date'].astype('str'), format='%d/%m/%Y')
# data.find_missing_dates(holidays)

# selected_columns = ['adj_close', 'vol', 'issues_detected']
# txn_data = data.data[selected_columns]
# txn_data.to_pickle(txn_filepath)

In [1]:
feeder = DataFeeder(headless=False)

symbols = ['AAA', 'A32', 'HAG', 'AGD']

vnd_data = feeder.multiprocessing(
    methods='vnd',
    symbols_array=symbols,
    records=30,
    workers=4
)

qfutils - INFO - Start validating symbols array...
qfutils - INFO - Symbols array validation done
quantfin - INFO - vnd - Joining results
quantfin - INFO - vnd - Process 0 - Query result appended to consolidated dataframe
quantfin - INFO - vnd - Process 1 - Query result appended to consolidated dataframe
quantfin - INFO - vnd - Process 2 - Query result appended to consolidated dataframe
quantfin - INFO - vnd - Process 3 - Query result appended to consolidated dataframe
quantfin - INFO - vnd - Updating vnd class dataframe...
quantfin - INFO - vnd - Appending results to result set...
quantfin - INFO - vnd - Results are added to result set


In [9]:
feeder = DataFeeder(driver_exe=None)
symbols = ['AAA']

vnd_data = feeder.vnd_get_data(symbols, records=10)

quantfin - INFO - VNDirect - vnd_get_data method is triggered
qfbrowsing - WARNING - Can't launch web driver with the given exe file
Traceback (most recent call last):
  File "c:\Users\baoloc.lam\1-LocLam\PythonProjects\vn_stock_trading\qfbrowsing.py", line 170, in __init__
    webdriver.Chrome.__init__(self, driver_exe, chrome_options=options)
  File "C:\Users\baoloc.lam\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\chrome\webdriver.py", line 73, in __init__
    self.service.start()
  File "C:\Users\baoloc.lam\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 76, in start
    stdin=PIPE)
  File "C:\Users\baoloc.lam\AppData\Local\Continuum\anaconda3\lib\subprocess.py", line 800, in __init__
    restore_signals, start_new_session)
  File "C:\Users\baoloc.lam\AppData\Local\Continuum\anaconda3\lib\subprocess.py", line 1148, in _execute_child
    args = list2cmdline(args)
  File "C:\Users\baoloc.lam\AppData\Local\Contin

ValueError: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
